In [ ]:
import os
import csv
from datetime import date, datetime
import yaml
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from experiment.toolkits.plotting_helper import *
from matplotlib import rc
from scipy.optimize import curve_fit
project_dir = Path().resolve().parents[1]
sys.path.insert(0, str(project_dir))
from time import sleep
import pickle
import datetime
from scipy.stats import linregress
import matplotlib.ticker as ticker
import experiment.toolkits.figure_formatting.formatter_2023_az as fmt
import experiment.toolkits.figure_formatting.production_quality_plots as prod_plot

In [ ]:
os.chdir(str(project_dir))

In [ ]:
base_cmd = str(project_dir) + "/bin/modules/image-processing-benchmarking/filtering-test"
def get_processing_time (N_t, kernel_size, reps):
    psf_bin_path = str(project_dir) + "/resources/psf/psf_" + str(N_t) + '_' + str(kernel_size) + ".bin"
    if not os.path.isfile(psf_bin_path):
        psf_pickle_path = str(project_dir) + "/resources/psf/psf_" + str(N_t) + '_' + str(kernel_size) + ".pickle"
        os.popen("python " + str(project_dir) + "/tools/generate-fake-psf.py" + " " + str(N_t) + " " + str(kernel_size) + " " + psf_pickle_path).read()
        os.popen("python " + str(project_dir) + "/tools/psf-translator.py --pickle " + psf_pickle_path + " --address " + psf_bin_path).read()
    command = base_cmd + " " + psf_bin_path + " " + str(N_t) + ' ' + str(kernel_size) + ' ' + str(reps) 
    return os.popen(command).read()

In [ ]:
reps = 1000
N_t = [8,16,32,64,128,256,512,1024,2048] 
kernel_size = [1, 3, 5, 7, 9]
results = [] 

for i in range(len(kernel_size)):
    print(f"{i}/{len(kernel_size)}")

    ks = kernel_size[i]
    temp = []
    for nt in N_t:
        acq_time_str = get_processing_time(nt, ks, reps)
        temp.append((float(acq_time_str.split('\n')[0]), float(acq_time_str.split('\n')[1])))
    results.append(temp)

# Save results

In [ ]:
# Saving the results 
file_name = datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
with open(os.path.join(str(project_dir), "resources", "image-processing-characterization", file_name), "wb") as file:
    pickle.dump(
        {
            "xvals": N_t,
            "kernels": kernel_size,
            "experiment": file_name,
            "data": results 
        }, file
        )

# Load results

In [ ]:
load_latest = True 
if not load_latest:
    file_name = "2024-06-13, 10-54-01.pickle"

with open(os.path.join(str(project_dir), "resources", "image-processing-characterization", file_name), "rb") as file:
    data = pickle.load(file)
    N_t = data["xvals"]
    kernel_size = data["kernels"]
    title = data["experiment"]
    results = data["data"]

In [ ]:
xvals = N_t 
kernel_1 = results[0]
kernel_3 = results[1]
kernel_5 = results[2]
kernel_7 = results[3]
result_1 = linregress(xvals, [x[0] * 1e6 for x in kernel_1])
result_3 = linregress(xvals, [x[0] * 1e6 for x in kernel_3])
result_5 = linregress(xvals, [x[0] * 1e6 for x in kernel_5])
result_7 = linregress(xvals, [x[0] * 1e6 for x in kernel_7])
print(result_1.intercept, result_1.slope)
print(result_3.intercept, result_3.slope)
print(result_5.intercept, result_5.slope)
print(result_7.intercept, result_7.slope)
fig, ax = plt.subplots() 
ax.set_xlabel("Number of traps")
ax.set_ylabel("Runtime (us)")
PlotPlotter(
    fig,
    ax,
    xvals,
    [x * result_1.slope + result_1.intercept for x in xvals],
    style = {"linestyle" : "-", "color": "#648EF7", "lw": 1}
).draw()
PlotPlotter(
    fig,
    ax,
    xvals,
    [x * result_3.slope + result_3.intercept for x in xvals],
    style = {"linestyle" : "-", "color": "#E0BB00", "lw": 1}
).draw()
PlotPlotter(
    fig,
    ax,
    xvals,
    [x * result_5.slope + result_5.intercept for x in xvals],
    style = {"linestyle" : "-", "color": "#57B52C", "lw": 1}
).draw()
PlotPlotter(
    fig,
    ax,
    xvals,
    [x * result_7.slope + result_7.intercept for x in xvals],
    style = {"linestyle" : "-", "color": "#C46BC7", "lw": 1}
).draw()
prod_plot.production_quality_scatter(fig, ax, [xvals, xvals, xvals, xvals], [[(x[0] * 1e6) for x in kernel_1], [(x[0] * 1e6) for x in kernel_3], [(x[0] * 1e6) for x in kernel_5], [(x[0] * 1e6) for x in kernel_7]], yerr_list=[[(x[1] * 1e6) for x in kernel_1], [(x[1] * 1e6) for x in kernel_3], [(x[1] * 1e6) for x in kernel_5], [(x[1] * 1e6) for x in kernel_7]], save_path=str(project_dir) + "/image-processing.pdf")
ax.legend(["1x1", "3x3", "5x5", "7x7"],fontsize=4)
prod_plot.yticks(ax, [0,60,120,180, 240])
prod_plot.xticks(ax, [0, 512, 1024, 1536, 2048])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.savefig("image-processing.pdf", format='pdf')
plt.show()

In [ ]:
def func(x, a, b, c):
  return  c * x ** 2 + a 

fig, ax = plt.subplots()
# Set the font size and style
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
fontsize = 7
plt.rc('pdf', fonttype=42) # make the test exportable as text, not an image

x_list = [[1],[3],[5],[7]]
y_list = [[kernel_1[7][0]*1e6], [kernel_3[7][0]*1e6], [kernel_5[7][0]*1e6], [kernel_7[7][0]*1e6]]
yerr_list=[[kernel_1[7][1]*1e6], [kernel_3[7][1]*1e6], [kernel_5[7][1]*1e6], [kernel_7[7][1]*1e6]]
ax.set_aspect('auto')
model, _ = curve_fit(func, [1,3,5,7], [kernel_1[7][0]*1e6, kernel_3[7][0]*1e6, kernel_5[7][0]*1e6, kernel_7[7][0]*1e6])
PlotPlotter(
    fig,
    ax,
    range(10),
    [x**2 * model[2] + model[0] for x in range(10)],
    style = {"linestyle" : "-", "color": "#A9A9A9", "lw": 1}
).draw()
# Iterate over each data set in y_list and with the corresponding lists from x_list scatter plot the scatter plot the data
# If xerr_list and yerr_list are not None, then include the corresponding error bars with the data
markers = ['o', 's', 'v', '>', '<', 'D', 'p', 'd']
palette = {
    'primary': ['#D9E3FD', '#FFF1AD', '#DAF3CE', '#EDD2EE', '#F6B6C7', '#FFD3AD', '#D9E3FD', '#FFF1AD',],
    'secondary':['#648EF7', '#E0BB00','#57B52C', '#C46BC7', '#DA1B4E', '#E06900', '#648EF7', '#E0BB00']
}
marker_size = 4
xerr_list = None
for i in range(len(y_list)):    
    ax.scatter(x_list[i], y_list[i], marker=markers[i], s=marker_size, linewidths = 0.75, 
                edgecolors=palette['secondary'][i], facecolors=palette['primary'][i], zorder = 2)
    
    if xerr_list != None:
        if yerr_list != None:
            ax.errorbar(x=x_list[i], y=y_list[i], xerr=xerr_list[i], yerr=yerr_list[i], 
                        linestyle='None', elinewidth=0.5, ecolor=palette['secondary'][i], zorder = 1)
        else:
            ax.errorbar(x=x_list[i], y=y_list[i], xerr=xerr_list[i], linestyle='None', 
                        elinewidth=0.5, ecolor=palette['secondary'][i], zorder = 1)
    else:
        if yerr_list != None:
            ax.errorbar(x=x_list[i], y=y_list[i], yerr=yerr_list[i], linestyle='None', 
                        elinewidth=0.5, ecolor=palette['secondary'][i], zorder = 1)
        else:
            ax.errorbar(x=x_list[i], y=y_list[i], xerr = xerr_list, yerr = yerr_list, linestyle='None', 
                        elinewidth=0.5, ecolor=palette['secondary'][i], zorder = 1)
    
figure_width_cm = 2.8
figure_height_cm = 2.8
figure_width_in = figure_width_cm / 2.54
figure_height_in = figure_height_cm / 2.54

fig.set_size_inches((figure_width_in, figure_height_in))  # Set the margins of our production quality figure
left_margin = (4.3-3.48-0.0235)/4.3
right_margin = (4.3-0.05)/4.3
top_margin = (2.8-0.05)/2.8
bottom_margin = (2.8-2.21-0.0235)/2.8

plt.subplots_adjust(
    left=left_margin,
    right=right_margin,
    bottom=bottom_margin,
    top=top_margin,
)

prod_plot.yticks(ax, [0,30,60,90, 120])
prod_plot.xticks(ax, [1,3, 5, 7])    
ax.set_title('')
 
# Format the ticks as desired for the new figure
ax.tick_params(axis='both', direction='in', length = 2, labelsize = fontsize, pad = 2, width = 0.75)
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')

xlim = ax.get_xlim()
ylim = ax.get_ylim()



tick_pad_x = 1.5
tick_pad_y = 1
ax.tick_params('x', labelsize=fontsize, pad=tick_pad_x) 
ax.tick_params('y', labelsize=fontsize, pad=tick_pad_y) 

# Formats the number of decimals in the axes ticks

# Formtats the axes labels
label_x = ax.get_xlabel()
label_y = ax.get_ylabel()
label_pad_x = 1
label_pad_y = 1
ax.set_xlabel(label_x, fontsize=fontsize, labelpad=label_pad_x)
ax.set_ylabel(label_y, fontsize=fontsize, labelpad=label_pad_y)

# Sets the xlim and ylim axes so the the ticks are not right to the end (creating the corner square))
epsilon_x = 0.05
epsilon_y = epsilon_x
ax.set_xlim(xlim[0] - epsilon_x * (xlim[1] - xlim[0]), xlim[1] + epsilon_x * (xlim[1] - xlim[0]))
ax.set_ylim(ylim[0] - epsilon_y * (ylim[1] - ylim[0]), ylim[1] + epsilon_y * (ylim[1] - ylim[0]))

# Control the line thickness for the axes
for spine in ax.spines.values():
    spine.set_linewidth(0.75)

ax.set_xlabel("Box dimension")
ax.set_ylabel("Runtime (us)")

plt.savefig("image-processing-square.pdf", format='pdf')
plt.show()